#NIRC2 preprocessing

We present here functions dedicated to Keck NIRC2 preprocessing. It consists in a basic preprocessing (masterflat), determination of the center of the VORTEX from sky or sci images, registration of the images and the creation of cube.


More specifically, this notebook talks about:
0. [Import](#import)
1. [Basic preprocessing](#prepro)
  0. [Open a fits file](#open)  
  1. [Master flat](#mflat)
  2. [Preprocess files](#prefiles)
  3. [Remove bad pixels](#badpix)
  4. [Create a cube from fits images](#cube)
2. [Find the VORTEX center from a sky image](#center)
   0. [Introduction](#intro)
   1. [Step-by-step procedure](#proc)
       1. [Initialization](#ini)
       2. [Define a model (optional)](#usermodel)
       3. [Minimization](#min)
       4. [Representation](#res)
   4. [Routine](#routine)
   5. [Evolution of the VORTEX position](#evo)
3. [Find the VORTEX center directly from sci images](#centerdust)   
4. [Registration](#regi)
5. [Parallactic angles](#paral)
    0. [PA for a set of images](#paral_set)
    1. [PA for a single image](#paral_single)
    2. [Calculating the PA directly from $\mathrm{DEC}$, $\phi$ and $H$](#paral_calc)
6. [Crop the cube](#crop)
7. [ADI PCA and RDI PCA: brief overview](#pca)

**Tips**
+ To obtain the complete docstring for a specific function, open a new cell and run: function_name?.
+ To take a look at the source code of a specific function, open a new cell and run: function_name??.

##Import <a id='import'></a>

We import all the functions from the module **NIRC2_Preprocessing**. Furthermore, we will make use of few VIP and numpy package functions. Whatever part of the notebook you plan to execute, you better first run this cell. 

In [ ]:
from NIRC2_Preprocessing import *
import numpy as np
from vip.fits import display_array_ds9, write_fits
from vip.calib.cosmetics_sdi import bad_pixel_removal

%matplotlib inline
%load_ext autoreload
%autoreload 2

---

##Basic preprocessing <a id='prepro'></a> 

###Open a fits file <a id='open'></a>

**Summary**: Open a fits file and display it with DS9. Optionally, the header is extracted.

We can easily open a fits image, even those which raise a *missing END card* error, by using the function **open_fits( )**. If the parameter **header** is set to *False* (default value), only 1 output is returned: image = open_fits(path); otherwise, 2 outputs are returned, respectively the image and the header.

In [ ]:
path = '/Users/Olivier/Documents/ULg/VORTEX/Data/RAW/Keck/HR8799/20150609/sci/sci_Tint_0p2_coadds_100/n0377.fits'
image, header = open_fits(path, header=True, verbose=False)

The variable **image** contains the fits image as a numpy array and can be displayed with DS9 using the VIP function **display_array_ds9( )**. As explained in the VIP docstring, DS9 should be already installed in your system along with XPA.

In [ ]:
print image.shape
display_array_ds9(image)

The variable **header** is a dictionary which contains all the fits header cards. 

In [ ]:
# Specific header card
card = 'PARANG'
print '{} = {}'.format(card,header[card])

## All header cards
#for key, value in header.items():
#    print '{} = {}'.format(key,value)
## Or
header

###Master flat <a id='mflat'></a>

**Summary**: create a master flat from a set of flat images. 

We define the repository which contains all flats.

In [ ]:
path_flat = '/Users/Olivier/Documents/ULg/VORTEX/Data/RAW/Keck/HR8799/20150610/flat/'

We list all the fits files using the function **listing( )**. If the parameter **selection** is set to *True*, each fits image will be opened with DS9 and you will be asked to keep it or discard it. If *False*, all fits images in the folder will be used to create the master flat.

In [ ]:
file_list = listing(path_flat, selection=False, ext='fits')

print ''
for filename in file_list:
    print filename

From the flat images listed in **file_list**, the master flat is created with the function **masterFlat( )**.

+ If **header** is *True*, a list of all individual headers associated to all the flat images is returned.
+ If **norm** is *True*, the master flat is normalized.
+ If **display** is *True*, the master flat is automatically displayed with DS9.
+ If **save** is *True*, the master flat is saved into the 'mflat.fits' file in the same folder as the individual flats.

In [ ]:
mflat, headers = masterFlat(file_list, header=True, norm=True, display=True, save=False)

From **headers**, one can retrieve informations about all the flats. For instance:

In [ ]:
for j in range(len(file_list)):
    print '{}: {} at {}'.format(file_list[j],headers[j]['DATE-OBS'],headers[j]['EXPSTART'])

###Preprocess files <a id='prefiles'></a>
**Summary**: divide a set of fits image by the master flat and save them into a folder. 

We define two paths which respectively contains: 
+ all the files to process
+ the master flat 

Then, we list all files in the files-to-process folder with the function **listing( )**.

In [ ]:
# Master flat
path_mflat = '/Users/Olivier/Documents/ULg/VORTEX/Data/RAW/Keck/Calibration/mflat_20150610.fits'

# (1) Folder which include a set of files, - OR -, (2) a single file:
##  (1) Repository (if you want to process a set of images all at once)
path_files = '/Users/Olivier/Documents/ULg/VORTEX/Data/RAW/Keck/HR8799/20150609/sci/sci_Tint_0p2_coadds_100/'
file_list = listing(path_files, selection=False, ext='fits')

## (2) Single file (if you want to process a single image)
#file_list = ['HR8799_Keck/20150609/sky/n0404.fits']

for filename in file_list:
    print filename

Then, by the use of the function **applyFlat( )**, we divide all images by the master flat and (optionally) save them into a specific folder. 

+ If **header** is *True*, a list of all individual headers associated to all the flat images is returned.
+ If **display** is *True*, the preprocessed images are automatically displayed with DS9.
+ If **save** is *True*, the preprocessed images are saved into the 'path_files/flatted/' repository.

In [ ]:
preprocessed, headers = applyFlat(file_list, path_mflat, header=True, display=False, save=True, verbose=True)

The variable **headers** is a dictionary which contains the headers of all preprocessed images. The variable **preprocessed** is a dictionary which contains the preprocessed images for a further possible use. We display a raw image and its associated preprocess one with DS9.  

In [ ]:
key = file_list[0]
image = open_fits(key, header=False, verbose=False)
display_array_ds9(image, preprocessed[key])
print 'Flat preprocessing for image: {}'.format(key)

###Remove bad pixels <a id='badpix'></a>

This section is dedicated to the image bad pixels removal. We use the vip function **bad_pixel_removal( )**. For more informations, please consult the vip docstring (>>> bad_pixel_removal?).

In [ ]:
# Path
path = '/Users/Olivier/Documents/ULg/VORTEX/Data/Cube_PSF_PA/Keck/HR8799/20150609/'
data = 'cube_HR8799_20150609.fits'
output_filename = 'cube_HR8799_20150609_CleanedUp_new.fits'

# Define the approximate y and x coordinates of the star where no bad pixel is corrected 
# in a circle of 2 fwhm (given by fwhm_round). 
center = np.array([296,296])

# other parameters
fwhm_round = 20
sig = 5.0

# Open the cube (or single image) we want to clean up
cube, header = open_fits(path+data, header=True, verbose=False)
cube_cu = cube.copy()

# Let's go
for j in range(cube_cu.shape[0]):
    print 'step {}/{}'.format(j+1,cube.shape[0])
    print ''
    frame = cube_cu[j,:,:].copy()
    res = bad_pixel_removal(frame, center, fwhm_round=fwhm_round, sig=sig, verbose=True)
    cube_cu[j,:,:] = res[1][0]
    print ''
    display_array_ds9(cube[j,:,:],cube_cu[j,:,:])

# Write the cleaned up cube
write_fits(path+output_filename, cube_cu, header=None, verbose=True)

# Display the cleaned up cube
display_array_ds9(cube, cube_cu)

print ''
print 'DONE !'

###Create a cube from fits images <a id='cube'></a>
**Summary**: create and save a cube from a set of fits images.

From a set of $N$ fits images (registered or not), we create a cube with the shape $N \times l \times c$ where $l \times c$ corresponds to the size of each image in pixels. 

+ If **header** is *True*, a list with all fits image headers is returned.
+ If **save** is *True*, the cube is saved into the current repository.

In [ ]:
path_files = '/Users/Olivier/Documents/ULg/VORTEX/Data/RAW/Keck/HR8799/20150609/sci/sci_Tint_0p2_coadds_100/flatted/'
file_list = listing(path_files, selection=False)

cube, headers = create_cube_from_frames(path_files, header=True, verbose=False, save=False)    

One can display the cube with DS9... 

In [ ]:
# Display the cube with DS9
display_array_ds9(cube)

or retrieve header cards of all original files:

In [ ]:
# Extract some header cards
dec = [h['DEC'] for h in headers]
ra = [h['RA'] for h in headers]
exp = [h['EXPSTART'] for h in headers]

for j, filename in enumerate(file_list):
    print '{}: [{},{}] at {}'.format(filename,ra[j],dec[j],exp[j])

---

##Find the VORTEX center from a sky image <a id='center'></a>

0. [Introduction](#intro)
1. [Step-by-step procedure](#proc)
   1. [Initialization](#ini)
   2. [Define a model (optional)](#usermodel)
   3. [Minimization](#min)
   4. [Representation](#res)
4. [Routine](#routine)
5. [Evolution of the VORTEX position](#evo)

###Introduction <a id='intro'></a> 
We model the VORTEX signature profile with a 2-D Gaussian profile. It constitutes so far a good approximation and should be sufficient to obtain the position of the center and reduce the effect due to the model choice. For this purpose, we minimize a function of merit which compare the intensity of all pixels in a squared box with the corresponding intensity obtained from the model. The function of merit is a reduced $\chi^2_r$ defined by:

$\chi^2_r = \frac{1}{N-n_p}\sum_{j=1}^{N} \frac{\left(I_j - I_j^{model}\right)^2}{I_j}$,

where $n_p$ is the number of model parameters and $N$ the total number of pixels in the box (if $L$ is the size of the box in pixels, we have $N = L^2$). The error associated to $I_j$ is $\sigma_j = \sqrt{I_j}$. The 2-D Gaussian profile is defined by:


$I^{Gaussian}(x,y) = \mbox{bkg} + I_0 \exp{\left[- \left(\frac{(x-x_0)^2}{2 \sigma_x^2} + \frac{(y-y_0)^2}{2 \sigma_y^2}\right)\right]}$,

where $\mbox{bkg}$ represents the background, $I_0$ the maximum intensity (taking account of the $\mbox{bkg}$) and $(x_0,y_0)$ the position where $I = I_0$. Other models are available, such as a cone or a Moffat profile defined by: 


$I^{Moffat}(x,y) = \mbox{bkg} + I_0 \left[1 + \left(\frac{(x-x_0)^2 + (y-y_0)^2}{\alpha^2}\right) \right]^{-\beta}$,
where $\alpha$ is a scale parameter and $\beta$ the parameter which determines the overall shape of the profile.

###Step-by-step procedure <a id='proc'></a> 

The following steps illustrate the procedure to determine the VORTEX center from a sky image.

####Initialization <a id='ini'></a> 
We first display the image with DS9 in order to roughtly estimage the position of the VORTEX center.

In [ ]:
path_file = '/Users/Olivier/Documents/ULg/VORTEX/Data/RAW/Keck/HR8799/20150609/sky/flatted/n0373_flatted.fits'

image = open_fits(path_file, header=False, verbose=False)
display_array_ds9(image)

We initialize the center position of a squared box and its size (in pixels). Then, we display a 3-D surface plot of the pixel intensities in the box by using the function **plot_surface** (for more information about this function: ?plot_surface in a new cell). 

In [ ]:
# Initializate the center and the size (in pixels) of the squared box
center, size = (521,731), 100 # VORTEX center 
# (521,731), 16 # VORTEX center
# (592,606), 31 # Dust

# Display a 3-D surface plot of the box
plot_surface(image, center, size, output=False, figsize=(16,14), cmap='jet')

We adopt a model for the VORTEX signature and define the associated additional parameters. This module already includes various models:
+ Gaussian profile, callable through **gauss2d( )**, 2 additional parameters = $\sigma_x$, $\sigma_y$
+ Gaussian symetrical profile, callable through **gauss2d_sym( )**, 1 additional parameter = $\sigma$
+ Moffat profile, callable through **moffat( )**, 2 additional parameters = $\alpha$, $\beta$
+ Cone profile, callable through **cone( )**, 1 additional parameter = radius

In [ ]:
fun = gauss2d
p_additional = [12,12]

Then, the vector which contains all the initial parameter values is created.

In [ ]:
# The box
box = image[center[0]-size//2:center[0]+size//2,center[1]-size//2:center[1]+size//2]

# The center of the box
x_ini, y_ini = (size//2,size//2) 

# The background estimation
bkg_ini = np.median(image) 

# The maximum intensity
i0_ini = np.max(box) - bkg_ini

# The initial values for all parameters to optimize
p_initial = np.array([x_ini,y_ini,i0_ini,bkg_ini]+p_additional)

####Define a model (optional) <a id='usermodel'></a>

If you have already adopted a model for the VORTEX signature, just skip this step and go to [Minimization](#min). 

Let us note that you can also define your own model. For instance, we adopt a Sersic profile defined by:

$I^{Sersic}(x,y) = \mbox{bkg} + I_0 \exp{\left[- \left(\frac{\sqrt{(x-x_0)^2 + (y-y_0)^2}}{\alpha}\right)^{1/n}\right]}$

Then, we only have to define a function **sersic( )**, and the corresponding **p_initial** vector. The only requirement is that the 2 first arguments must the (x,y) grid. Indeed, these arguments are fixed the others will be optimized during the minimization procedure. However, we advocate to organize the arguments as follow: $x, y, x_0, y_0, I_0, \mbox{bkg}$, [additional parameters] where 
+ $(x_0,y_0)$ locates the position of the maximum intensity
+ $I_0$ is the maximum intensity irrespectively of the background, i.e. $I_0 = I(x_0,y_0) - \mbox{bkg}$
+ $\mbox{bkg}$ is the background

For the case of the Sersic profile, there are 2 additional parameters, respectively $\alpha$ and $n$. Finally, the code is given by: 

In [ ]:
# Define the Sersic profile as a callable function
def sersic(x, y, x0, y0, i0, bkg, alpha, n):
    r = ((x-x0)**2+(y-y0)**2)**0.5
    return bkg + i0 * np.exp(-(r/(alpha))**(1/float(n)))

# *fun* is passed to vortex_center() function. But, we can directly pass sersic or any other.
fun = sersic
p_additional = [20, 0.7]

# Define the vector which contains the initial values for all parameters, i.e. x0, y0, i0, bkg, [additional parameters]
box = image[center[0]-size//2:center[0]+size//2,center[1]-size//2:center[1]+size//2]
x_ini, y_ini = (size//2,size//2) 
bkg_ini = np.median(image)
i0_ini = np.max(box) - bkg_ini

p_initial = np.array([x_ini,y_ini,i0_ini,bkg_ini]+p_additional)

####Minimization <a id='min'></a> 

To start the minimization, we call the function **vortex_center( )**. The first 5 arguments are mandatory while the others are optional, including:

+ If **Display** is *True*, some figures are displayed during the minimization.
+ If **verbose** is *True*, the results are displayed at the end of the minimization.
+ ****kwargs** which are options passed to scipy.optimize.minimize( ). For instance, solver_options = {'xtol': ...}.

The **vortex_center( )** function returns a tuple of 3 objects:
1. the position of the VORTEX center in the original image
2. the information returned by the minimization tool
3. the box grid as a tuple of 2 numpy.array (useful to represent the model)

In [ ]:
solver_options = {'xtol': 1e-04, 'maxiter': 1e+05, 'maxfev': 1e+05}
center_vor, minimization_output, grid = vortex_center(image, 
                                                      center, 
                                                      size, 
                                                      p_initial,
                                                      fun,
                                                      display= True, 
                                                      verbose=True,
                                                      savefig=False,
                                                      method = 'Nelder-Mead',
                                                      options = solver_options)

p_optimized = minimization_output.x

####Representation <a id='res'></a> 

In [ ]:
print 'Representation of the best model (in terms of chi2)'
print '---------------------------------------------------'
print 'Model adopted: {}'.format(fun)

model = fun(grid[0],grid[1],*p_optimized)
plot_surface(model, figsize=(12,10), cmap='jet')

###Routine <a id='routine'></a> 

Here is a routine which allows to determine the center of the VORTEX for a set of raw images. Into a loop, each image is optionally preprocessed (flat) and the position is determined (Nelder-Mead minimization). 

Parameters initialization. The **cards** parameter allows to extract specific header cards from all the images. It can be useful when you want to [determine the evolution of the position of the VORTEX center in function of time](#evo) or pointing position, ...

In [ ]:
# Repositories which contain all files to process (or a list of file paths) and, if required, the master flat.
path_files = '/Users/Olivier/Documents/ULg/VORTEX/Data/RAW/Keck/HR8799/20150608/sky/Tint_0p2_coadds_100/flatted/'
path_mflat = None

# Box parameters
center, size = (521,731), 100
# (521,731), 16 # VORTEX center
# (592,606), 31 # Dust

# Model and additional parameter(s), depending on which model we've adopted
fun = gauss2d_sym
p_model = [12]

# Routine parameters
preprocess = False
cards = ['EXPSTART','DATE-OBS','RA','DEC']
verbose = True

Let's go !

In [ ]:
center_all, success_all, file_list, header_cards = vortex_center_routine(path_files, 
                                                                         center, 
                                                                         size, 
                                                                         fun,
                                                                         preprocess=preprocess, 
                                                                         path_mflat=path_mflat,
                                                                         additional_parameters=p_model, 
                                                                         verbose=verbose,
                                                                         cards=cards,
                                                                         options={'xtol':1e-04, 'maxiter':1e+08,' maxfev':1e+08})

print ''
print 'Convergence reached for all minimization ? {}'.format(success_all)

Informations exctracted from the headers.

In [ ]:
for k, filename in enumerate(file_list):
    print '{} = [{:.2f},{:.2f}]'.format(filename,header_cards['RA'][k], header_cards['DEC'][k])

###Evolution of the VORTEX position <a id='evo'></a> 

From the routine outputs, we can represent the position of the VORTEX center as a function of time. For this aim, we use the function **timeExtract( )** as well as the matplotlib.pyplot module.

In [ ]:
# From the observation date and time, we determine the delta time between the first and all other observations
t_start = timeExtract(header_cards['DATE-OBS'],header_cards['EXPSTART'])
delta_time = [(t-t_start[0]).seconds/3600. for t in t_start]

# Then, we illustrate the obtained VORTEX position as a function of time
import matplotlib.pyplot as plt
plt.figure(figsize=(14,7))
plt.hold('on')
plt.plot(delta_time,center_all[:,0]-center_all[0,0],'.r', markersize=14)
plt.xlabel(r'$\Delta t$ (hour)',fontsize=20)
plt.ylabel(r'$\Delta x$ (pixels)',fontsize=20)
plt.title(path_files)
#plt.ylim([-0.03,0.06])
#plt.savefig(path_files+'DUST_delta_x_'+'Gaussian'+'_'+path_files.split('/')[:-1][-1]+'.pdf')
plt.show()

plt.figure(figsize=(14,7))
plt.hold('on')
plt.plot(delta_time,center_all[:,1]-center_all[0,1],'.b', markersize=14)
plt.xlabel(r'$\Delta t$ (hour)',fontsize=20)
plt.ylabel(r'$\Delta y$ (pixels)',fontsize=20)
plt.title(path_files)
#plt.ylim([-0.86,-0.74])
#plt.savefig(path_files+'DUST_delta_y_'+'Gaussian'+'_'+path_files.split('/')[:-1][-1]+'.pdf')
plt.show()

---

##Find the VORTEX center directly from sci images <a id='centerdust'></a>

In the case of a dust signature can be seen on all sci images taken during the same run, we assume that its relative position with respect to the VORTEX center is constant. Then, if a sky image is available, we can determine the position of the VORTEX center for all the sci images by:
+ determining the center of the dust signature for all sci images
+ determining the VORTEX and dust centers in the sky image
+ deducing the relative position of the dust with respect to the VORTEX center
+ determining the position of the VORTEX center for all sci images

This procedure can take advantage of what we have already illustrated so far. 

Center of the dust signature in all sci images.

In [ ]:
path_sci_files = '/Users/Olivier/Documents/ULg/VORTEX/Data/RAW/Keck/HR8799/20150609/sci/sci_Tint_0p2_coadds_100/flatted/'
center, size = (592,606), 31
fun = gauss2d
p_model = [5,5]
center_all_dust, success_all, file_list = vortex_center_routine(path_sci_files, 
                                                             center, 
                                                             size, 
                                                             fun,
                                                             preprocess=False, 
                                                             path_mflat=None,
                                                             additional_parameters=p_model, 
                                                             verbose=False,
                                                             cards=None)

# Illustration of the dust signature in the first image of file_list
image = open_fits(file_list[0], header=False, verbose=False)
plot_surface(image, center, size, output=False, figsize=(8,6), cmap='jet')

Centers of the VORTEX and dust signature in a sky image.

In [ ]:
path_sky_files = '/Users/Olivier/Documents/ULg/VORTEX/Data/RAW/Keck/HD219196/20150609/sky/flatted/n0373_flatted.fits'
sky_image = open_fits(path_sky_files, header=False, verbose=False)
display_array_ds9(sky_image)

# Dust center
center_dust, size_dust = (592,606), 31
fun_dust = gauss2d
p_model_dust = [5,5]

center_dust, success_dust, _ = vortex_center_routine(path_sky_files, 
                                                     center_dust, 
                                                     size_dust, 
                                                     fun_dust,
                                                     preprocess=False, 
                                                     path_mflat=None,
                                                     additional_parameters=p_model_dust, 
                                                     verbose=False,
                                                     cards=None)

# VORTEX center
center_vortex, size_vortex = (520,730), 30
fun_vortex = gauss2d
p_model_vortex = [5,5]

center_vortex, success_vortex, _ = vortex_center_routine(path_sky_files, 
                                                         center_vortex, 
                                                         size_vortex, 
                                                         fun_vortex,
                                                         preprocess=False, 
                                                         path_mflat=None,
                                                         additional_parameters=p_model_vortex, 
                                                         verbose=False,
                                                         cards=None)

print 'Center of the dust signature: {}'.format(center_dust)
print 'Center of the VORTEX signature: {}'.format(center_vortex)

Relative position.

In [ ]:
relative_position = center_vortex[0]-center_dust[0]
print 'Relative position: {}'.format(relative_position)

The position of the VORTEX in all sci frames.

In [ ]:
center_all = center_all_dust + np.tile(relative_position,(center_all_dust.shape[0],1))
#center_all = center_all+np.random.uniform(-0.5,0.5,center_all.shape)
print center_all

---

##Registration<a id='regi'></a> 

Since the previous sections ([Find the VORTEX center](#center) or [Find the VORTEX center directly from sci images](#centerdust)), we are now able to determine the VORTEX center position. The next stage consists in registering a sequence of images with respect to the corresponding VORTEX center. To this end, we use the function **registration( )** which requires, at least, the 3 following parameters:
+ **fileList** : the list of paths to the N sci images to register.
+ **initial_position** : a N x 2 array which defines the N positions to register.
+ **final_position** : a 1 x 2 array which defines the position where all initial_position are registered in the processed cube.

We choose the **initial_position** to be the positions of the VORTEX center and the **final_position** to be the center of the frame.

###VORTEX center <a id='regivor'></a>

First, we have to determine the center of the VORTEX signature for a set of images. For this aim, select between [Routine](#routine) or [Find the VORTEX center directly from sci images](#centerdust) sections. At the end, we should have the variable **center_all**.

###Create the **initial_position** array <a id='regiini'></a>

If you have determined the position of the center of the VORTEX from a dust signature on all sci images, the matrix **center_all** and the list **file_list** should respectively contain all the positions of the VORTEX center and all the sci image paths. Then, you can skip this step and go directly to the next sub-section [Registration](#regireg).

If you have determined the position from few sky images, you have likely less VORTEX center positions than sci images. Then, you have to construct the **center_all** matrix in such a way that center_all.shape = (N,2) where N corresponds to the number of sci images to register. The next cell is dedicated to that part of the job. Some manual adjustments can be done here.

In [ ]:
# Path: images to register
path_files = '/Users/Olivier/Documents/ULg/VORTEX/Data/RAW/Keck/HR8799/20150609/sci/sci_Tint_0p2_coadds_100/flatted/' 
file_list = listing(path_files)
#for filename in file_list:
#    print filename

# Initial position from the VORTEX center position
part_0 = np.tile(center_all[0,:],(10,1))
part_1 = np.tile(center_all[1,:],(len(file_list)-10,1)) # np.array([520.06,730.5])

#part_0 = np.tile(center_from_sky[0,:],(9,1))
#part_1 = np.tile(center_from_sky[1,:],(10+10,1))#(len(file_list)-9,1))
#part_2 = np.tile(np.array([520.06,730.5]),(len(file_list)-9-10-10,1))

# Concatenation
center_all = np.concatenate((part_0,part_1))

 
print 'Number of files: {},  center_all shape: {}'.format(len(file_list),center_all.shape)

###Registration <a id='regireg'></a>

After defining the **final_position**, we run the **registration( )** function. It returns the cube which contains all registered images and (optional) their headers. 

In [ ]:
# Let's define the pixel coordinates at which all the VORTEX center will be shifted
final_position = np.array([512,512])

# Registration
cube_reg, headers = registration(file_list, 
                                 initial_position = center_all, 
                                 final_position = final_position, 
                                 header=True, 
                                 verbose=False, 
                                 display=True,
                                 save=True)

---

##Parallactic angles <a id='paral'></a> 

###PAs of a set of images <a id='paral_set'></a>

For a set of images, the parallactic angles can be retreived from few header cards:
+ *ROTPPOSN*: the rotator physical position
+ *PARANTEL*: the parallactic angle telescope
+ *EL*: the telescope elevation
+ *INSTANGL*: porg to instrument angle

combined as follows (Justin Crepp):

parallactic angle = *ROTPPOSN* + *PARANTEL* - *EL* - *INSTANGL* .

In [ ]:
# Define the folder which contains the images
root_to_images = '/Users/Olivier/Documents/ULg/VORTEX/Data/RAW/Keck/HR8799/20150609/sci/sci_Tint_0p2_coadds_100/flatted/'

# Let's go
file_list = listing(root_to_images, selection=False, ext='fits')
parallactic_angles = np.zeros(len(file_list))

for k, filename in enumerate(file_list):
    _, header = open_fits(filename, header=True)
    parallactic_angles[k] = header['ROTPPOSN']+header['PARANTEL']-header['EL']-header['INSTANGL']    
    
print parallactic_angles

We save the parallactic angles into a 1-column fits file.

In [ ]:
pa_filename = '/Users/Olivier/Documents/ULg/VORTEX/Data/Cube_PSF_PA/Keck/HR8799/20150609/pa_HR8799_20150609.fits' 
write_fits(pa_filename,np.array(parallactic_angles))

###PA of a single image <a id='paral_single'></a>

The same idea but for only one image.

In [ ]:
data = '/Users/Olivier/Documents/ULg/VORTEX/Data/RAW/Keck/HR8799/20150609/sci/sci_Tint_0p2_coadds_100/n0377.fits'
_, header = open_fits(data, header=True)
parallactic_angle = header['ROTPPOSN']+header['PARANTEL']-header['EL']-header['INSTANGL']

print 'PA = {}'.format(parallactic_angle)

###Calculating the PA directly from $\delta$, $\phi$ and $H$. <a id='paral_calc'></a>

From the header, one can retreived informations about the declinaison $\delta$, observation date and hour angle $H$ at the given observation. Given the geographical latitude $\phi$ of the observer, one can deduce the parallactic angle $q$ from (Meeus, 1998, Eq. 13.1 p94):

$\tan{q} = \frac{\sin{H}}{\tan{\phi} \cos{\delta} - \sin{\delta} \cos{H}}$ .

The function **get_parang( )** performs that calculation and requires:
+ the header
+ the geographical latitude of the observer
+ the epoch of the declinaison written in the header (default=None). 

If **epoch** is None, the header card *EQUINOX* is used if it exists, otherwise 'FK5' is used. The parameter **epoch** can also accept 'FK5', 'todate' or any epoch (float). In the first and third cases, the declinaison is precessed from initial epoch to observation epoch. Furthermore, if **epoch** is not None, the header card *EQUINOX* is not used.


In [ ]:
data = '/Users/Olivier/Documents/ULg/VORTEX/Data/RAW/Keck/HR8799/20150609/sci/sci_Tint_0p2_coadds_100/n0377.fits'
_, header = open_fits(data, header=True)
parallactic_angle = get_parang(header, latitude=19.82636, epoch=None)

print 'PA = {}'.format(parallactic_angle)

---

##Crop the cube<a id='crop'></a> 

This part is dedicated to crop the cube of registered frames. To this end, we use the function **cube_crop_frames_optimized( )** which requires at leat the 3 following parameters:
+ cube : the cube to crop
+ ceny : the y-coordinate of the center of the cropped cube
+ cenx : the x-coordinate of the center of the cropped cube

The optimized size of the cube frames are automatically determined in such a way it maximizes the area of non-zero pixel values.

In [ ]:
cube_path = '/Users/Olivier/Documents/ULg/VORTEX/Data/RAW/Keck/HR8799/20150609/sci/sci_Tint_0p2_coadds_100/flatted/reg/cube_reg.fits'
cube_reg = open_fits(cube_path)

cropped_cube_path = '/Users/Olivier/Documents/ULg/VORTEX/Data/Cube_PSF_PA/Keck/HR8799/20150609/cube_HR8799_20150609.fits'

final_position = np.array([512,512])
cube_reg_crop = cube_crop_frames_optimized(cube_reg, 
                                           final_position[1], final_position[0], 
                                           ds9_indexing=True, 
                                           verbose=True, 
                                           display=True,
                                           save=True,
                                           filename=cropped_cube_path)

---

## ADI PCA and RDI PCA: brief overview <a id='pca'></a>

We propose here a basic call of VIP functions to perform ADI or RDI PCA of the obtained cube and have a quick look to the result. A more in-depth analyze can be found in the **Photo_Astrometry.ipynb** IPython notebook, including PCA, Vortex center determination, Speckle noise and MCMC, ...

Open the files

In [ ]:
path_cube = '/Users/Olivier/Documents/ULg/VORTEX/Data/Cube_PSF_PA/Keck/HR8799/20150609/cube_HR8799_20150609_CleanedUp_new.fits'
path_ref = '/Users/Olivier/Documents/ULg/VORTEX/Data/Cube_PSF_PA/Keck/HR8799/20150609/cube_HD219196_20150609.fits'
path_pa = '/Users/Olivier/Documents/ULg/VORTEX/Data/Cube_PSF_PA/Keck/HR8799/20150609/pa_HR8799_20150609.fits'

cube = open_fits(path_cube, header=False, verbose=False)
cube_ref = open_fits(path_ref, header=False, verbose=False)
angs = open_fits(path_pa, header=False, verbose=False)

Display the cube(s)

In [ ]:
display_array_ds9(cube)
#display_array_ds9(cube, cube_ref)

Process

In [ ]:
from vip.pca import pca
out_ADI = pca(cube, angs, cube_ref=None, ncomp=2, svd_mode='randsvd', full_output=False)
out_RDI = pca(cube, angs, cube_ref=cube_ref, ncomp=1, svd_mode='randsvd', full_output=False, center='global')

Display the result(s)

In [ ]:
display_array_ds9(out_ADI)
#display_array_ds9(out_ADI,out_RDI)

Write the resulting image into a fits file.

In [ ]:
filename = '/Users/Olivier/Documents/ULg/VORTEX/Data/Cube_PSF_PA/Keck/HR8799/20150609/HR8799_ADI_20150609_Cleanedup_new.fits'
write_fits(filename,out_ADI)

##Work in progress...

---